# 内容说明
1. 实验目标  
算法内容：借助SGD随机梯度下降，手动实现 MF 矩阵分解的推荐系统算法  
数据来源：使用 RecBole 中提供的 ml-100k 数据  
误差标准：使用 RMSE 均方根误差作为误差的衡量标准  
  
2. 本ipynb内容  
编写 MF 类，包括训练与测试  
考虑了论文中提到的三种偏置，包括 mu, bu 和 bi。  
代码编写完成后，进行调参与对比实验

3. 训练集与测试集划分  
没有将整个的评分矩阵 R 作为类的参数，因为不太好根据矩阵 R 划分训练和测试集  
先随机打乱评分文件，共100000条数据，4/5作为训练数据，1/4作为测试数据，提前构造好训练的评分矩阵和测试的评分矩阵作为参数传入MF类中

In [1]:
import pandas as pd
import torch
from torch.distributions import normal

## 1. MF模型
1. init()函数：使用正态分布N(0,0.1)来初始化各个参数，训练集和测试集在调用模型前划分好，传入类的是训练和测试的评分矩阵。可调参数包括：迭代次数epochs，嵌入空间维数embed_size，学习率gamma，学习率衰减系数weight_decay，正则化系数lmbda。
2. compute_mu()函数：根据训练集计算平均评分值，在SGD_train()函数训练前调用，计算好self.mu
3. compute_loss()函数：计算每轮迭代后的损失值，便于验证是否沿着下降的趋势，也可以判断何时停止迭代
4. SGD_train()函数：使用SGD随机梯度下降更新参数
5. test()函数：使用训练好的参数在测试集上测试，并计算RMSE和MAE数值。
6. see()函数：返回训练好的各个参数，不然不好直接访问，以防想要计算其它的一些东西

In [2]:
class MF():
    
    def __init__(self, R_train, R_test, epochs, embed_size=10, gamma=0.05, weight_decay=0.8, lmbda=0.05):
        # 各个参数
        self.epochs = epochs  # 迭代次数
        self.embed_size = embed_size  # 映射的空间维度
        self.gamma = gamma  # 学习率
        self.weight_decay = weight_decay  # 学习率衰减
        self.lmbda = lmbda  # 正则化系数
        
        # 初始化训练和测试的评分矩阵以及P, Q参数矩阵
        self.R_train = R_train  # 用于训练的评分矩阵
        self.R_test = R_test  # 用于测试的评分矩阵
        self.n_P = R_train.size()[0]  # user数目
        self.n_Q = R_train.size()[1]  # item数目
        m = normal.Normal(0.0, 0.1)  # 均值为0，标准差为0.1的正态分布
        self.P = m.sample((self.n_P, embed_size))  # 初始化user参数矩阵
        self.Q = m.sample((self.n_Q, embed_size))  # 初始化item参数矩阵
        
        # 偏置
        self.mu = 0  # 评分均值
        self.bu = m.sample((self.n_P, 1))
        self.bi = m.sample((self.n_Q, 1))
        
    
    # 计算训练集的评分均值 mu
    def compute_mu(self):
        n_R = 0  # 非零元素个数，即已经评分的个数
        sum_R = 0  # 评分总和
        for u in range(1, self.n_P):
            for i in range(1, self.n_Q):
                rui = self.R_train[u][i]
                if rui != 0:
                    n_R += 1
                    sum_R += rui
        self.mu = sum_R / n_R
    
    
    # 定义计算损失值的函数，用于判断每次迭代是否有效
    def compute_loss(self):
        
        # 提前计算好参数的二范数，不然在循环中可能重复计算
        bu_norm = torch.norm(self.bu)**2
        bi_norm = torch.norm(self.bi)**2
        P_norm = 0  # 总的user向量范数
        Q_norm = 0  # 总的item向量范数
        for u in range(1, self.n_P):
            pu = self.P[u]
            P_norm += torch.norm(pu)**2
        for i in range(1, self.n_Q):
            qi = self.Q[i]
            Q_norm += torch.norm(qi)**2
        loss = self.lmbda*(P_norm + Q_norm + bu_norm + bi_norm)  # 初始化误差
        
        # 遍历评分矩阵，计算平均误差
        n = 0
        for u in range(1, self.n_P):
            pu = self.P[u]  # 提取该用户的f维向量
            for i in range(1, self.n_Q):
                rui = self.R_train[u][i]  # 真实评分
                if rui != 0:  # 判断是否有评分
                    n += 1
                    qi = self.Q[i]  # 提取该物品的f维向量
                    rui_pred = sum(pu*qi) + self.mu + self.bu[u] + self.bi[i]
                    eui = rui - rui_pred  # 真实评分与预测评分的偏差
                    loss = loss + eui**2  # 累加误差
        return loss/n
    

    # 使用 SGD 随机梯度下降训练参数
    def SGD_train(self):
        loss_list = []  # 存储每轮迭代后的总误差
        self.compute_mu()  # 计算好评分均值mu
        
        # 计算初始误差值
        loss = self.compute_loss()
        print('初始平均误差为 ', loss)
        loss_list.append(loss)
        
        # 多次迭代
        for k in range(self.epochs):
            # 遍历评分，根据每个 rui，按照 SGD 更新参数
            for u in range(1, self.n_P):
                for i in range(1, self.n_Q):
                    rui = self.R_train[u][i]
                    if rui != 0:  # 判断是否已评分
                        pu = self.P[u]
                        qi = self.Q[i]
                        rui_pred = sum(pu*qi) + self.mu + self.bu[u] + self.bi[i]
                        eui = rui - rui_pred  # 用更新后的来计算误差，因为SGD是对每个样本都计算更新
                
                        # 更新参数
                        self.P[u] += self.gamma * (eui*qi - self.lmbda*pu)
                        self.Q[i] += self.gamma * (eui*pu - self.lmbda*qi)
                        self.bu[u] += self.gamma * (eui - self.lmbda*self.bu[u])
                        self.bi[i] += self.gamma * (eui - self.lmbda*self.bi[i])
            self.gamma *= self.weight_decay  # 学习率逐渐减小
            
            # 一次迭代完成后，计算平均误差并测试性能
            loss = self.compute_loss()
            loss_list.append(loss)
            rmse, mae = self.test()
            print('epoch %d: avg_loss=%.4f, RMSE=%.4f, MAE=%.4f' % (k+1, loss, rmse, mae))
        return loss_list
        
        
    # 在测试集上预测，并计算RMSE和MAE性能
    def test(self):
        # 预测结果
        R_pred = torch.matmul(self.P, self.Q.t()) + self.mu + self.bu + self.bi.t()
        
        # 计算 RMSE 和 MAE
        rmse = 0  # 初始化均方根误差
        mae = 0  # 初始化平均绝对误差
        n = 0  # 测试集已评分个数
        # 遍历每个评分
        for u in range(1, self.n_P):
            for i in range(1, self.n_Q):
                rui = self.R_test[u][i]
                rui_pred = R_pred[u][i]
                if rui != 0:
                    n += 1
                    rmse += (rui - rui_pred)**2
                    mae += torch.abs(rui - rui_pred)
        rmse = torch.sqrt(torch.tensor([rmse/n]))
        mae /= n
        
        return rmse, mae
    
    
    # 返回各个参数矩阵
    def see(self):
        return self.P, self.Q, self.mu, self.bu, self.bi

## 2. 数据预处理
1. 读取评分文件
2. 随机打乱数据
3. 划分训练集和测试集，9/10作为训练集，1/10作为测试集，并建立相应的评分矩阵

In [8]:
# 读取评分文件
inter = pd.read_csv('../dataset/ml-100k/ml-100k.inter', delimiter='\t', engine='python')

# 创建评分矩阵，分为训练矩阵和测试矩阵
n_user = 943
n_item = 1682
R_train = torch.Tensor(n_user+1, n_item+1).zero_()
R_test = torch.Tensor(n_user+1, n_item+1).zero_()
print(R_train.size())
print(R_test.size())

# 填充训练集和测试集的评分矩阵值
# 先随机打乱数据
df = pd.DataFrame(inter)
df = df.sample(frac=1).reset_index(drop=True)
# 前4/5作为训练数据
df1 = df[:80000]
for index, row in df1.iterrows():
    R_train[row['user_id:token']][row['item_id:token']] = row['rating:float']
# 后1/5作为测试数据
df2 = df[80000:100000]
for index, row in df2.iterrows():
    R_test[row['user_id:token']][row['item_id:token']] = row['rating:float']

torch.Size([944, 1683])
torch.Size([944, 1683])


## 3. 运行与测试 
在MF模型中，参数P, Q, bu和bi都是按照正态分布N(0,0.1)初始化的   
可以调整参数，寻求最佳结果

In [9]:
# epochs=15, embed_size=16, gamma=0.05, weight_decay=1, lmbda=0.05

mf = MF(R_train, R_test, epochs=20, embed_size=16, gamma=0.005, weight_decay=1, lmbda=0.05)
loss_list = mf.SGD_train()  # 训练

初始平均误差为  tensor([1.2959])
epoch 1: avg_loss=1.0046, RMSE=1.0099, MAE=0.8163
epoch 2: avg_loss=0.9382, RMSE=0.9813, MAE=0.7834
epoch 3: avg_loss=0.9069, RMSE=0.9686, MAE=0.7697
epoch 4: avg_loss=0.8876, RMSE=0.9611, MAE=0.7621
epoch 5: avg_loss=0.8741, RMSE=0.9562, MAE=0.7571
epoch 6: avg_loss=0.8640, RMSE=0.9527, MAE=0.7535
epoch 7: avg_loss=0.8559, RMSE=0.9501, MAE=0.7508
epoch 8: avg_loss=0.8493, RMSE=0.9481, MAE=0.7487
epoch 9: avg_loss=0.8435, RMSE=0.9465, MAE=0.7471
epoch 10: avg_loss=0.8383, RMSE=0.9451, MAE=0.7457
epoch 11: avg_loss=0.8335, RMSE=0.9440, MAE=0.7445
epoch 12: avg_loss=0.8289, RMSE=0.9431, MAE=0.7435
epoch 13: avg_loss=0.8243, RMSE=0.9422, MAE=0.7426
epoch 14: avg_loss=0.8197, RMSE=0.9413, MAE=0.7417
epoch 15: avg_loss=0.8149, RMSE=0.9405, MAE=0.7409
epoch 16: avg_loss=0.8099, RMSE=0.9397, MAE=0.7400
epoch 17: avg_loss=0.8046, RMSE=0.9388, MAE=0.7391
epoch 18: avg_loss=0.7988, RMSE=0.9379, MAE=0.7382
epoch 19: avg_loss=0.7926, RMSE=0.9370, MAE=0.7373
epoch 20: avg_